## Download Dataset 

Today we are using the open dataset from '今日頭條'(https://www.toutiao.com/)
There are 382688 titles along with their categories in this dataset. One article belongs to one category. Repository: https://github.com/skdjfla/toutiao-text-classfication-dataset

In [ ]:
%%bash 
cd /home/ec2-user/SageMaker/ 
mkdir chinese-corpus
cd chinese-corpus
git clone https://github.com/skdjfla/toutiao-text-classfication-dataset.git 
cp toutiao-text-classfication-dataset/toutiao_cat_data.txt.zip .
unzip toutiao_cat_data.txt.zip 


In [ ]:
!head -n 10 /home/ec2-user/SageMaker/chinese-corpus/toutiao_cat_data.txt

### Observe the number of documents over each category 

In [ ]:
cat /home/ec2-user/SageMaker/chinese-corpus/toutiao_cat_data.txt | awk -F '_!_' '{print $3}' | sort | uniq -c | sort -k1n

### Check if there are multi-labeled documents 

In [ ]:
cat /home/ec2-user/SageMaker/chinese-corpus/toutiao_cat_data.txt | awk -F '_!_' '{print $1}' | sort | uniq -c | sort -k1n | tail -10  

### Install necessary packages 

In [ ]:
!pip install jieba
!pip install ckiptagger 
!pip install tensorflow==1.13.1

In [ ]:
%%bash 
wget -O data.zip https://tinyurl.com/yb6mnvb8
unzip  data.zip

In [ ]:
import sys
sys.path.append('/home/ec2-user/SageMaker/nlp_processing/')

from preprocess import TouTiaoNewsPreprocessor
from segmenter import JiebaSegmenter
from transformer import BlazingTextInputDataTransformer

preprocessor = TouTiaoNewsPreprocessor()
res = preprocessor.preprocess('/home/ec2-user/SageMaker/chinese-corpus/toutiao_cat_data.txt')
print('result of preprocess:', res[0:10])
segmenter = JiebaSegmenter()
transformer = BlazingTextInputDataTransformer()
all_contents = [] 
for r in res:
    toks = segmenter.segment(r[2])
    input = transformer.transform((r[0], r[1], toks))
    all_contents.append(input)    
print('result of transformed data:', all_contents[0:10])    

In [ ]:
import pickle 
pickle.dump( all_contents, open( "/home/ec2-user/SageMaker/chinese-corpus/all_contents.p", "wb" ) )

In [ ]:
from segmenter import CKIPSegmenter
ckipsegmenter = CKIPSegmenter()
ckipsegmenter.segment('歡迎大家來亞馬遜上課！')


In [ ]:
segmenter.segment('歡迎大家來亞馬遜上課！')